**Consolidated Monthly Rapport for DK**

This code generates the Monthly Rapport for DK

In [ ]:
# Define the file path for Shipments Out
# Replace the path with the actual path to your Shipments Consolidated file
shipments = shipments.read_csv(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\Shipments_Consolidated.csv')

# Define the file path for Goods In
# Replace the path with the actual path to your Inbound Orderline file
inbound = inbound.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Inbound Order Line.xlsx')

# Define the file path for Moved to Production
# Replace the path with the actual path to your Production file
production = production.read_csv(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\Production_Consolidated.csv')

# Define the file path for Inbound from Production
# Replace the path with the actual path to your Inbound from Production file
inbound_prod = inbound_prod.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Inbound from Production.xlsx')

# Define the file path for Inventor Ultimo
# Replace the path with the actual path to your Inventory Consolidated file
inventory = inventory.read_csv(r'')

SyntaxError: unterminated string literal (detected at line 3) (1039494101.py, line 3)

In [ ]:
# Groupby to create summary tables
goods_in = inbound.groupby("Varenummer")["Mængde modtaget"].sum().reset_index()
moved_to_prod = production.groupby("ItemNumber")["QuantityProduced"].sum().reset_index()
inbound_from_prod = inbound_prod.groupby("ItemNumber")["QuantityPacked"].sum().reset_index()
shipments_out = shipments.groupby("ItemNumber")["QuantityPacked"].sum().reset_index()

In [ ]:
# Merge all summary tables into a single report

report = goods_in.merge(moved_to_prod, on="SKU", how="outer") \
                 .merge(inbound_from_prod, on="SKU", how="outer") \
                 .merge(shipments_out, on="SKU", how="outer")

In [5]:
import pandas as pd
from pathlib import Path

# --- File paths (edit if needed) ---
base = Path(r"C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup")

shipments_path = base / "Shipments_Consolidated.csv"
inbound_order_path = base / "2025-09-02T03_44_52_230_inboundOrderLine (3).xlsx"
production_path = base / "Production_Consolidated.csv"
inbound_from_prod_path = base / "(ITA) Inbound from Production.xlsx"

# --- Load files ---
shipments = pd.read_csv(shipments_path)

# inbound order line (Excel, single sheet: 'inboundOrderLine')
inbound = pd.read_excel(inbound_order_path, sheet_name="inboundOrderLine", engine="openpyxl")

# production consolidated (CSV)
production = pd.read_csv(production_path)

# inbound from production (Excel, two sheets: DK + DE) -> concat
inbound_prod_dk = pd.read_excel(inbound_from_prod_path, sheet_name="Aug25 Prod Order-DK", engine="openpyxl")
inbound_prod_de = pd.read_excel(inbound_from_prod_path, sheet_name="Aug25 Prod Order - DE", engine="openpyxl")
inbound_prod = pd.concat([inbound_prod_dk, inbound_prod_de], ignore_index=True)

# --- Aggregations (SUMIF-by-SKU) ---
goods_in = (
    inbound.groupby("Varenummer", dropna=False)["Mængde modtaget"]
    .sum()
    .reset_index()
    .rename(columns={"Varenummer": "SKU", "Mængde modtaget": "Goods In"})
)

moved_to_production = (
    production.groupby("ItemNumber", dropna=False)["QuantityProduced"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityProduced": "Moved to Production"})
)

inbound_from_production = (
    inbound_prod.groupby("ItemNumber", dropna=False)["QuantityPacked"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityPacked": "Inbound from Production"})
)

shipments_out = (
    shipments.groupby("ItemNumber", dropna=False)["QuantityPacked"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityPacked": "Shipments Out"})
)

# --- Merge into one report ---
report = (
    goods_in
    .merge(moved_to_production, on="SKU", how="outer")
    .merge(inbound_from_production, on="SKU", how="outer")
    .merge(shipments_out, on="SKU", how="outer")
)

# Fill missing numeric values
for col in ["Goods In", "Moved to Production", "Inbound from Production", "Shipments Out"]:
    if col in report.columns:
        report[col] = report[col].fillna(0)

# Optional extra columns (remove if you only want the four sums)
report["In Production"] = report["Moved to Production"] - report["Inbound from Production"]  # WIP proxy
report["Serial no."] = pd.NA   # placeholder
report["Variance"] = 0         # placeholder
report["Inventory ULTIMO"] = report["Goods In"] + report["Inbound from Production"] - report["Shipments Out"]

# Choose/Order columns
cols = [
    "SKU", "Goods In", "Moved to Production", "In Production",
    "Inbound from Production", "Shipments Out", "Serial no.",
    "Variance", "Inventory ULTIMO"
]
report = report[[c for c in cols if c in report.columns]].sort_values("SKU", kind="stable")

# Save
out_path = base / "SKU_Flow_Report.csv"
report.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"Saved to: {out_path}")


Saved to: C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\SKU_Flow_Report.csv
